In [1]:
import torch
from transformers import AutoTokenizer
import numpy as np
from sagemaker.serve.spec.inference_spec import InferenceSpec

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


# Testing Loading of the Compiled Model 
This is the model that was deployed to AWS using the model's torch.jit.trace .pth object. In this notebook the model is loaded using torch.jit.load which is what AWS uses in their PyTorch image container environment. This model is large so it cannot be uploaded using the AWS sagemaker GUI so instead, the model artifacts where saved in a AWS S3 bucket and the path used as the input for the model in sagemaker's deployable model action. The huggingface environment variable HF_TASK for text-generation was added and then tested by scanning the logs using AWS cloudwatch. Model loading errors occured due to other issues (does not exist when loading in this notebook). **The model was deployed to an AWS container using huggingface endpoint's GUI using the repo containing the finetuned model. Note that the model's 'generate' method used to generate text was not traced due to untracable modules used in the generate method.**

In [2]:
model = torch.jit.load('./serve/Story-Generation-Model.pt')

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Alexis-Az/Story-Generation-Model")

In [4]:
DUMMY_TEXT = "Hi! Can you tell me a story about a cute park?"

In [5]:
class InferenceSpec(InferenceSpec):
    def invoke(self, input_object: object, model: object):       
        with torch.no_grad():
            output = model(input_object, True)
        return output
        
    def load(self, model_dir: str):
        model = torch.jit.load(model_dir+f"/{INFER_MODEL.split('/')[-1]}.pt")
        model.eval()
        return model

inf_spec = InferenceSpec()

In [6]:
%%capture
model.eval()

In [7]:
tokens = tokenizer(DUMMY_TEXT, return_tensors='pt')

In [8]:
tokens

{'input_ids': tensor([[128000,  13347,      0,   3053,    499,   3371,    757,    264,   3446,
            922,    264,  19369,   6246,     30]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [15]:
output

tensor([[[ 2.1406,  5.0000,  9.0625,  ..., -1.6094, -1.6094, -1.6094],
         [15.6250,  3.7969,  3.3125,  ..., -4.2188, -4.2188, -4.2188],
         [ 3.9062,  0.8047,  6.2500,  ..., -1.7656, -1.7656, -1.7656],
         ...,
         [ 7.8438,  3.3438,  3.7188,  ..., -4.9062, -4.9062, -4.9062],
         [12.2500,  5.5312,  5.2188,  ..., -2.9062, -2.9062, -2.9062],
         [ 7.5000, -0.4863,  4.7188,  ..., -1.8359, -1.8359, -1.8359]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<ToCopyBackward0>)